In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import tqdm
from tqdm.auto import trange, tqdm
import matplotlib.pyplot as plt
from matplotlib import image

import deepface
from deepface import DeepFace
import gradio as gr

In [84]:
models = ['VGG-Face', 'Facenet', 'Facenet512', 'openFace', 'DeepFace', 'DeepId', 'ArcFace', 'Dlib', 'SFace']
backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']

result = DeepFace.verify(
    img1_path = 'records/CYS-18-5884/1721675648162.jpg',
    img2_path = 'records/CYS-18-8479/1721675646498.jpg',
    model_name = models[1],
    distance_metric = 'cosine',
    enforce_detection = False,
    detector_backend = backends[0],
    align = False,
    threshold = .2
)

In [85]:
result

{'verified': False,
 'distance': 0.8278660258791868,
 'threshold': 0.2,
 'model': 'Facenet',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 0,
   'y': 0,
   'w': 1459,
   'h': 2210,
   'left_eye': None,
   'right_eye': None},
  'img2': {'x': 0,
   'y': 0,
   'w': 1664,
   'h': 2639,
   'left_eye': None,
   'right_eye': None}},
 'time': 8.36}

In [186]:
mat_no = list()
for i in tqdm(os.listdir("records")):
    mat_no.append("/".join(i.split("-")))


img_path = list()
for i in tqdm(os.listdir("records")):
    record_ = f"records/{i}"
    rec_ = list()
    for n, j in enumerate((os.listdir(record_))):
        rec_.append(f"records/{i}/{j}")
    img_path.append(" ".join(rec_))  

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [187]:
len(img_path), len(mat_no)

(15, 15)

In [188]:
records_df = pd.DataFrame(
    data = {
        "matric number": mat_no,
        "img paths": img_path
    }
)
records_df.head()
records_df.to_csv("records.csv", index=0)

In [189]:
# simple python workflow

mat_no_ = input("Input Matric No here(DEPT/YY/NNNN).....")
student_name = input("Input your name here.......")

df = pd.read_csv("records.csv")
mat_nos = [i for i in df["matric number"].values]

if mat_no_ in mat_nos:
    print("Matric number found in Database")
    verified = True
else:
    print("Student matric number not found in database")
    verified = False

Matric number found in Database


In [190]:
if verified:
    df_sort = df[df["matric number"] == mat_no_]

In [191]:
df_sort

,matric number,img paths
0,CYS/18/5874,records/CYS-18-5874/1721676589154.jpg records/...


In [192]:
imgs_ = df_sort["img paths"].values[0]
imgs = imgs_.split(" ")
imgs

['records/CYS-18-5874/1721676589154.jpg',
 'records/CYS-18-5874/1721676589194.jpg']

In [197]:
img_ = image.imread('records/CYS-18-8472/1721675648545.jpg')
# img_

In [198]:
verify_status = list()
for img in imgs:
    result = DeepFace.verify(
        img1_path = img_,
        img2_path = img,
        model_name = models[1],
        distance_metric = 'cosine',
        enforce_detection = False,
        detector_backend = backends[0],
        align = False,
        threshold = .2
    )

    verify_status.append(result["verified"])

In [206]:
verify_status.index(False)

0

In [9]:
def image_predict(mat_no_, student_name, img_):
    models = ['VGG-Face', 'Facenet', 'Facenet512', 'openFace', 'DeepFace', 'DeepId', 'ArcFace', 'Dlib', 'SFace']
    backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']
    df = pd.read_csv("records.csv")
    mat_nos = [i for i in df["matric number"].values]

    if mat_no_ in mat_nos:
        verified = True
    else:
        verified = False
    
    if verified:
        df_sort = df[df["matric number"] == mat_no_]
        imgs_ = df_sort["img paths"].values[0]
        imgs = imgs_.split(" ")

        verify_status = list()
        for img in imgs:
            result = DeepFace.verify(
                img1_path = img_,
                img2_path = img,
                model_name = models[1],
                distance_metric = 'cosine',
                enforce_detection = False,
                detector_backend = backends[0],
                align = False,
                threshold = .2
            )

        verify_status.append(result["verified"])

        if True in verify_status:
            response_ = f"{student_name} verified"
            img_match_id = imgs.index(True)
            img_match = imgs[img_match_id]
            img_match = gr.Image(value = "no_img.jpg")

            return response_, img_match
        else:
            response_ = f"{student_name} not verified(does not match image in the Database)"
            img_match = gr.Image(value = img_)
            return response_, img_match
    else:
        response__ = f"{student_name} not found in Database"
        img_match = gr.Image(value = "no_img.jpg")
        
        return response__, img_match
    


with gr.Blocks() as demo:
    m_no = gr.Textbox(label = "Input Matric No here(DEPT/YY/NNNN).....")
    name_ = gr.Textbox(label = "Input your name here.......")
    image_ = gr.Image(label = 'Input Image to be predicted', source = "webcam")
    output1 = gr.Textbox(label = 'Response')
    output2 = gr.Image()
    btn = gr.Button('Verify')
    btn.click(fn = image_predict, inputs = [m_no, name_, image_], outputs = [output1, output2])

demo.launch(share = False)

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Ayo Agbaje\my_venv_\lib\site-packages\gradio\routes.py", line 422, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Users\Ayo Agbaje\my_venv_\lib\site-packages\gradio\blocks.py", line 1326, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "c:\Users\Ayo Agbaje\my_venv_\lib\site-packages\gradio\blocks.py", line 1260, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "c:\Users\Ayo Agbaje\my_venv_\lib\site-packages\gradio\components.py", line 1868, in postprocess
    raise ValueError("Cannot process this value as an Image")
ValueError: Cannot process this value as an Image
